In [ ]:
! pip install pylibjpeg pylibjpeg-libjpeg pydicom
! pip install -U python-gdcm

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1 Libraries</b></p>
</div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
import gdcm
import pydicom
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
import scipy.ndimage
from tqdm import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings
import dask.array as da
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, callbacks
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image as im
from keras.models import load_model
from random import shuffle
import random
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>2 Activating Devices</b></p>
</div>

In [ ]:
DEVICE = "GPU"
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>3 Data</b></p>
</div>


In [ ]:
#For Segmentation data
def load_NIfTI(path):
    mask = nib.load(path)
    
    # Convert to numpy array
    seg = mask.get_fdata()
    
    # Align orientation with images
    seg = seg[:, ::-1, ::-1].transpose(2, 1, 0)
    
    return seg

In [ ]:
#Getting patient with mask
seg_paths = glob(f"../input/rsna-2022-cervical-spine-fracture-detection/segmentations/*")
training_patient=[]
for path in seg_paths:
    training_patient.append((path.rsplit("/",1)[-1])[:-4])#Patient with mask present

In [ ]:
#Example segment image
path_mask=f"../input/rsna-2022-cervical-spine-fracture-detection/segmentations/{training_patient[0]}.nii"
patient_mask=load_NIfTI(path_mask)

patient_mask.shape

In [ ]:
# Plot segment images
fig, axes = plt.subplots(nrows=3, ncols=6, figsize=(24,12))
fig.suptitle(f'ID: {training_patient[0]}', weight="bold", size=20)

start=110
for i in range(start,start+18):
    mask = patient_mask[i]
    slice_no = i

    # Plot the image
    x = (i-110) // 6
    y = (i-110) % 6

    axes[x, y].imshow(mask, cmap='bone')
    axes[x, y].set_title(f"Slice: {slice_no}", fontsize=14, weight='bold')
    axes[x, y].axis('off')

In [ ]:
#Loading Scans
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Load the scans in given folder path
def load_scan(path):
    
    dcm_paths = glob(f"{path}/*")
    dcm_paths.sort(key=natural_keys)
    
    patient_scan = [pydicom.dcmread(paths) for paths in dcm_paths]
    
    return patient_scan

In [ ]:
#Example Scan
path_scan=f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/{training_patient[0]}"
image=load_scan(path_scan)

In [ ]:
# Plot images
fig, axes = plt.subplots(nrows=3, ncols=6, figsize=(24,12))
fig.suptitle(f'ID: {training_patient[0]}', weight="bold", size=20)

start = 110
for i in range(start,start+18):
    img = image[i].pixel_array
    slice_no = i

    # Plot the image
    x = (i-start) // 6
    y = (i-start) % 6

    axes[x, y].imshow(img, cmap="bone")
    axes[x, y].set_title(f"Slice: {slice_no}", fontsize=14, weight='bold')
    axes[x, y].axis('off')

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4 Preprocessing</b></p>
</div>


<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.1 Loding and Conversion to HU</b></p>
</div>

In [ ]:
def get_pixels_hu(slices):
   
    image = np.stack([cv2.resize(s.pixel_array,(512,512),interpolation = cv2.INTER_NEAREST) for s in slices])
    
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    image = da.from_array(image) #Using Dask to speed up processing
    
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image <= -1000] = 0
    
    # Convert to Hounsfield units (HU)
        
    intercept = da.from_array([slices[slice_number].RescaleIntercept for slice_number in range(len(slices))])
    slope = da.from_array([slices[slice_number].RescaleSlope for slice_number in range(len(slices))])
    
    intercept=intercept.reshape((-1,1,1))
    slope=slope.reshape((-1,1,1))
    
    image= slope * image.astype("float64")
        
    image+= intercept
     
    return image.astype("int16")

In [ ]:
patient_slice=get_pixels_hu(image)

In [ ]:
#Ploting pixel array
plt.imshow(image[110].pixel_array,cmap='bone')
plt.axis("off")

In [ ]:
#Ploting pixel array distribution
plt.hist(image[110].pixel_array.flatten(),color="r",bins=50)
plt.xlabel("Pixel Values")
plt.ylabel("Fequency")
plt.show()

In [ ]:
#Ploting HU array
plt.imshow(patient_slice[110],cmap='bone')
plt.axis("off")

In [ ]:
#Ploting HU distribution
plt.hist(patient_slice[110].flatten().compute(),color="r",bins=50)
plt.xlabel("HU Values")
plt.ylabel("Fequency")
plt.show()

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.2 Normalization</b></p>
</div>

As HU values range from **150** to **2050**. So we will use this values for normalization.

In [ ]:
MIN_BOUND = 150.0
MAX_BOUND = 2050.0
    
def normalize(image):
    image = (image - MIN_BOUND)*255.0 / (MAX_BOUND - MIN_BOUND)
    image[image>255] = 255.
    image[image<0] = 255.
    return image

In [ ]:
image=normalize(patient_slice)

In [ ]:
plt.imshow(image[110],cmap="bone")
plt.axis("off")

In [ ]:
#Delete all unused objects to free up memory
del patient_slice
del image
del fig
del axes
del path_scan
del path_mask
del patient_mask
del seg_paths

gc.collect()

<div style="color:white;display:fill;
            background-color: #A020F0;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>5 Model</b></p>
</div>

In [ ]:
#Segreggate the preprocessed image in these folders based on Segmentation data in Training Folder.
def segregate_Train(start,X_train_patient):
    
    train_ds_x=[]
    train_ds_y=[]
    for i in range(start,start+5):
        patient_ID=X_train_patient[i]
        
        patient_seg=load_NIfTI(f"../input/rsna-2022-cervical-spine-fracture-detection/segmentations/{patient_ID}.nii")
        
        patient_scan=load_scan(f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/{patient_ID}")
        patient_hu=get_pixels_hu(patient_scan)
        patient_hu_normalised=normalize(patient_hu)
        
        for j in tqdm(range(0,len(patient_seg))):
            classes=np.unique(patient_seg[j])
        
            temp_lables=np.zeros(9)
            for k in classes:
                if int(k)!=0:
                    if int(k)<8:
                        temp_lables[int(k)]=1
                    else:
                        temp_lables[8]=1
                else:
                    temp_lables[0]=1
            
            train_ds_x.append(patient_hu_normalised[j].astype(np.uint8))
            train_ds_y.append(temp_lables.astype(np.uint8))
        
    return train_ds_x,train_ds_y

In [ ]:
#Segreggate the preprocessed image in these folders based on Segmentation data in Training Folder.
def segregate_Val(start,X_Val_patient):
    
    val_ds_x=[]
    val_ds_y=[]
    for i in range(start,start+2):
        patient_ID=X_Val_patient[i]
        
        patient_seg=load_NIfTI(f"../input/rsna-2022-cervical-spine-fracture-detection/segmentations/{patient_ID}.nii")
        
        patient_scan=load_scan(f"../input/rsna-2022-cervical-spine-fracture-detection/train_images/{patient_ID}")
        patient_hu=get_pixels_hu(patient_scan)
        patient_hu_normalised=normalize(patient_hu)
        
        for j in tqdm(range(0,len(patient_seg))):
            classes=np.unique(patient_seg[j])
        
            temp_lables=np.zeros(9)
            for k in classes:
                if int(k)!=0:
                    if int(k)<8:
                        temp_lables[int(k)]=1
                    else:
                        temp_lables[8]=1
                else:
                    temp_lables[0]=1
            
            val_ds_x.append(patient_hu_normalised[j].astype(np.uint8))
            val_ds_y.append(temp_lables.astype(np.uint8))
        
    return val_ds_x,val_ds_y

In [ ]:
#Defining the model
def model_CNN():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (512,512,1)),
        tf.keras.layers.Conv2D(filters = 8, kernel_size = (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.30),
        tf.keras.layers.Conv2D(filters = 64, kernel_size = (7,7),padding = 'Same', activation ='relu'),
        tf.keras.layers.Conv2D(filters = 8, kernel_size = (5,5), activation ='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(4, 4)),
        tf.keras.layers.Dropout(0.30),
        tf.keras.layers.Conv2D(filters = 8, kernel_size = (7,7),padding = 'Same', activation ='relu'),
        tf.keras.layers.Conv2D(filters = 64, kernel_size = (5,5), activation ='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(6, 6)),
        tf.keras.layers.Dropout(0.30),
        tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', activation ='relu'),
        tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), activation ='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(3, 3)),
        tf.keras.layers.Dropout(0.30),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(12,activation="ReLU"),
        tf.keras.layers.Dense(9, activation='softmax')
    ])
    model.build([None, 512, 512, 1])
    print(model.summary())

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
    
    return model

In [ ]:
X_train_patient, X_Val_patient = train_test_split(training_patient,train_size=70,test_size=17,shuffle=True)

In [ ]:
def fit(Fold,model):
    shuffle(X_train_patient)
    shuffle(X_Val_patient)
    
    train_ds_x,train_ds_y=segregate_Train(5*Fold,X_train_patient)
    val_ds_x,val_ds_y=segregate_Val(2*Fold,X_Val_patient)
    
    #Convert into specific format from tensorflow model
    train_ds_x=da.array(train_ds_x)
    train_ds_y=da.array(train_ds_y)
    val_ds_x=da.array(val_ds_x)
    val_ds_y=da.array(val_ds_y)
    
    train_ds_x=np.reshape(train_ds_x,(-1,512,512,1))
    val_ds_x=np.reshape(val_ds_x,(-1,512,512,1))
    
    #Data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    datagen.fit(train_ds_x)
    data=datagen.flow(train_ds_x,train_ds_y)
    
    early_stopping = callbacks.EarlyStopping(
        min_delta=0.001, # minimium amount of change to count as an improvement
        patience=5, # how many epochs to wait before stopping
        restore_best_weights=True,
    )
    
    model.fit(data,epochs=100,shuffle=True,callbacks=[early_stopping],validation_data=(val_ds_x,val_ds_y))
    
    if os.path.exists("./Classifier.h5"):
        os.remove("./Classifier.h5")
    model.save("Classifier.h5")

In [ ]:
for i in range(0,15):
    if os.path.exists("./Classifier.h5"):
        keras.backend.clear_session()
        fit(random.randint(0,7),keras.models.load_model("./Classifier.h5"))
    else:
        fit(random.randint(0,7),model_CNN())